# Example simple schedule

At the moment this only works with one patient class.  But it can be adapted for multiple classes.

In [2]:
import pandas as pd
import numpy as np
import simpy

## Dummy schedule data

In [12]:
# generate random data
# this will be 1 class of patients over 5 days.

SEED = 42
PATIENT_CLASSES = 1
DAYS = 5
rng = np.random.default_rng(SEED)

# use numpy to gen and cast to df
schedule = pd.DataFrame(rng.integers(1,15 , size=(DAYS, PATIENT_CLASSES)))
schedule

,0
0,2
1,11
2,10
3,7
4,7


## Simulation code.

`PatientProcess` don't worry too much about the implementation.  Its just an delay using a triangular dist as an example.
Time is in minutes

In [8]:
class PatientProcess:
    '''
    Encapsulates the process a patient undergoes
    '''
    def __init__(self, identifier, env, servers):
        '''
        Constructor method
        
        Params:
        -----
        identifier: int
            a numeric identifier for the patient.
            
        env: simpy.Environment
            the simulation environment
            
        servers: simpy.Resource
            servers
        '''
        self.identifier = identifier
        self.env = env
        self.servers = servers
        
    def service(self):
        '''
        simualtes the service process for a call operator
        
        1. request and wait for a call operator
        2. phone triage (triangular)
        3. exit system
        '''
        # record the time that call entered the queue
        start_wait = env.now

        # request an operator 
        with self.servers.request() as req:
            yield req
            
            # record the waiting time to be seen
            self.waiting_time = self.env.now - start_wait
            print(f'patient {self.identifier} seen at ' \
                      + f'{self.env.now:.3f}')
            
            # sample call duration.
            call_duration = np.random.triangular(left=0.01, mode=0.12, 
                                                 right=0.16)
            yield self.env.timeout(call_duration)
            
            print(f'patient {self.identifier} service ended {self.env.now:.3f}; ' \
                      + f'waiting time was {self.waiting_time:.3f}')
    

## Scheduled Generator.

This is the main bit.  It just incrememts time in days and looks up how many patients should arrive.  
A for loop then creates the processes.

If you had multiple patient classes (for different types of operations etc.) then you could split the numbers over columns in a df.  Then you would loop on each column and create the appropraite patient process class (or function). You might need to think about priority etc in there if thats built into you model.

In [36]:
def scheduled_arrivals_generator(env, servers, schedule, delay):
    '''
    scheduled patients each day

    Parameters:
    ------
    env: simpy.Environment

    servers: simpy.Resource
        servers
        
    schedule: pd.DataFrame
        scheduled of patients
        
    delay: int
        Time in minutes between days
    '''

    patient_count = 0
    for day in range(len(schedule)):
        
        print(f'***** Start of Day {day}')

        # use iat to reference by row, col (be careful you get these the right way round!)
        n_arrivals = schedule.iat[day, 0]
        print(f'{n_arrivals} arrivals scheduled')
              
        # loop through patients of this class.
        for i in range(n_arrivals):
            new_patient = PatientProcess(patient_count, env, servers)
            env.process(new_patient.service())
            # do this manually
            patient_count += 1
            
        # you'd need to think through how to deal with multiple patient classes.  
        # could be you do this column by column just generate the processes in turn.
        
        # then I assume you want this event to happen at the start of each working day?
        # so delay this loop by that time e.g. 24 hours.
        yield env.timeout(delay)

## Script to run model

In [38]:
# model parameters
RUN_LENGTH = 6.0 * 1440 # you will need to go longer than the no. days scheduled to get all patients
N_SERVERS = 1
DELAY = 1440

# create simpy environment and operator resources
env = simpy.Environment()
servers = simpy.Resource(env, capacity=N_SERVERS)

# create a model
env.process(scheduled_arrivals_generator(env, servers, schedule, DELAY))
env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

***** Start of Day 0
2 arrivals scheduled
patient 0 seen at 0.000
patient 0 service ended 0.038; waiting time was 0.000
patient 1 seen at 0.038
patient 1 service ended 0.090; waiting time was 0.038
***** Start of Day 1
11 arrivals scheduled
patient 2 seen at 1440.000
patient 2 service ended 1440.082; waiting time was 0.000
patient 3 seen at 1440.082
patient 3 service ended 1440.178; waiting time was 0.082
patient 4 seen at 1440.178
patient 4 service ended 1440.315; waiting time was 0.178
patient 5 seen at 1440.315
patient 5 service ended 1440.432; waiting time was 0.315
patient 6 seen at 1440.432
patient 6 service ended 1440.541; waiting time was 0.432
patient 7 seen at 1440.541
patient 7 service ended 1440.604; waiting time was 0.541
patient 8 seen at 1440.604
patient 8 service ended 1440.730; waiting time was 0.604
patient 9 seen at 1440.730
patient 9 service ended 1440.885; waiting time was 0.730
patient 10 seen at 1440.885
patient 10 service ended 1440.945; waiting time was 0.885
p